In [3]:
import pandas as pd

In [4]:
data= pd.read_csv('medical_new.csv') #mention the right medical_new path 
data.head()

,Unnamed: 0,medical_specialty,transcription,medical_label,word2vec_embed,fasttext_label
0,3,cardiovascular_pulmonary,d m mode leave atrial enlargement left atrial ...,0,[-3.9130253e-01 1.6925055e+00 -9.7060895e-01 ...,__label__cardiovascular_pulmonary d m mode lea...
1,4,cardiovascular_pulmonary,left ventricular cavity size wall thickness ap...,0,[-5.44876158e-01 1.30335271e+00 -1.53507555e+...,__label__cardiovascular_pulmonary left ventric...
2,7,cardiovascular_pulmonary,d echocardiogram multiple view heart great ves...,0,[-0.40531728 0.8317133 -1.0865903 0.296900...,__label__cardiovascular_pulmonary d echocardio...
3,9,cardiovascular_pulmonary,description normal cardiac chamber size normal...,0,[-0.3478441 0.9546592 -1.0535976 1.201930...,__label__cardiovascular_pulmonary description ...
4,11,cardiovascular_pulmonary,d study mild aortic stenosis widely calcify mi...,0,[ 0.27329454 1.1477208 -0.88355297 1.073992...,__label__cardiovascular_pulmonary d study mild...


In [ ]:
pip install tensorflow_text

In [6]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [7]:
from tensorflow import keras

In [8]:
#bert layers
encoder_url= "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
preprocess_url= "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

In [36]:
bert_preprocess= hub.KerasLayer(preprocess_url)
bert_encoder= hub.KerasLayer(encoder_url)

In [51]:
# input layer -> bert preprocess -> bert encoder
text_input= keras.layers.Input(shape= (), dtype= tf.string, name= "text")
preprocess_text= bert_preprocess(text_input)
output= bert_encoder(preprocess_text)

In [52]:
#neural network

hidden_layer1 = keras.layers.Dense(128, activation='relu')(output['pooled_output'])
hidden_layer2 = keras.layers.Dense(64, activation='relu')(hidden_layer1)
dropout_layer = keras.layers.Dropout(0.1)(hidden_layer2)
output_layer = keras.layers.Dense(5, activation='sigmoid', name='ans')(dropout_layer)


In [53]:
#final model
model= keras.Model(inputs= [text_input], outputs= [output_layer])

In [54]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 keras_layer_2 (KerasLayer)  {'input_word_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_type_ids': (None,                                            
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             }                                                              

In [55]:
model.compile(
    optimizer= 'adam',
    loss='sparse_categorical_crossentropy',
    metrics= ['accuracy']
)

In [56]:
from sklearn.model_selection import train_test_split

In [57]:
x_train, x_test, y_train, y_test= train_test_split(data['transcription'], data['medical_label'], test_size= 0.2, stratify= data['medical_label'], random_state= 5354)

In [58]:
model.fit(x_train, y_train, epochs= 50, batch_size= 8)

Epoch 1/50
120/120 [==============================] - 18s 96ms/step - loss: 1.6189 - accuracy: 0.2132
Epoch 2/50
120/120 [==============================] - 12s 96ms/step - loss: 1.5774 - accuracy: 0.2727
Epoch 3/50
120/120 [==============================] - 12s 100ms/step - loss: 1.5538 - accuracy: 0.3020
Epoch 4/50
120/120 [==============================] - 12s 100ms/step - loss: 1.5018 - accuracy: 0.3260
Epoch 5/50
120/120 [==============================] - 12s 101ms/step - loss: 1.4629 - accuracy: 0.3406
Epoch 6/50
120/120 [==============================] - 13s 104ms/step - loss: 1.4466 - accuracy: 0.3595
Epoch 7/50
120/120 [==============================] - 12s 103ms/step - loss: 1.3912 - accuracy: 0.3908
Epoch 8/50
120/120 [==============================] - 12s 103ms/step - loss: 1.3807 - accuracy: 0.3971
Epoch 9/50
120/120 [==============================] - 12s 100ms/step - loss: 1.3327 - accuracy: 0.4033
Epoch 10/50
120/120 [==============================] - 12s 96ms/step - loss

In [59]:
model.evaluate(x_test, y_test)

8/8 [==============================] - 4s 371ms/step - loss: 0.8778 - accuracy: 0.6375


[0.8777868747711182, 0.637499988079071]

In [60]:
a= model.predict(["The intricate interplay of the cardiovascular and pulmonary systems orchestrates the rhythm of life itself. The heart's ceaseless pump propels oxygen-rich blood through a vast network of vessels, sustaining every cell. Meanwhile, the lungs breathe life into this blood, exchanging life-sustaining oxygen for carbon dioxide waste. This delicate dance of oxygenation and circulation fuels our vitality, and the fields of cardiovascular and pulmonary medicine are dedicated to preserving this intricate balance. From preventing heart disease to diagnosing respiratory disorders, these disciplines hold the keys to sustaining a life of vigor and wellness"])

1/1 [==============================] - 4s 4s/step


In [61]:
a[0][0]

0.9697437

In [62]:
y_pred= model.predict(x_test)

8/8 [==============================] - 4s 432ms/step


In [64]:
import numpy as np
y_pred_lab= [np.argmax(i) for i in y_pred]

In [65]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_lab))

              precision    recall  f1-score   support

           0       0.71      0.73      0.72        56
           1       0.72      0.74      0.73        39
           2       0.53      0.26      0.35        34
           3       0.69      0.82      0.75        61
           4       0.45      0.48      0.47        50

    accuracy                           0.64       240
   macro avg       0.62      0.61      0.60       240
weighted avg       0.63      0.64      0.63       240

